In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as func

In [22]:
spark = SparkSession.builder.appName("FirstApp").getOrCreate()

In [15]:
myschema = StructType([\
                       StructField("userID", IntegerType(), True),
                       StructField("name", StringType(), True),
                       StructField("age",IntegerType(), True),
                       StructField("friends",IntegerType(), True),
                        ])

In [16]:
people = spark.read.format("csv")\
    .schema(myschema)\
    .option("Path","fakefriends.csv")\
    .load()

In [29]:
people.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)



In [17]:
output = people.select(people.userID,people.name\
                       ,people.age,people.friends)\
         .where(people.age < 30).withColumn('insert_ts', func.current_timestamp())\
         .orderBy(people.userID)

In [18]:
output.count()

112

In [44]:
filtered_people = people.filter(func.col("friends") > 400)
filtered_people.show(10)

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     3|  Deanna| 40|    465|
|    13|Jean-Luc| 56|    444|
|    18|Jean-Luc| 45|    455|
|    25|     Ben| 21|    445|
|    28|  Martok| 49|    476|
|    40|     Odo| 52|    413|
|    61|  Kasidy| 62|    442|
|    66|  Geordi| 21|    477|
|    75|    Morn| 40|    459|
|    77|  Weyoun| 40|    407|
+------+--------+---+-------+
only showing top 10 rows



In [46]:
age_group = people.groupBy("age").count()
age_group.show(10)

+---+-----+
|age|count|
+---+-----+
| 31|    8|
| 65|    5|
| 53|    7|
| 34|    6|
| 28|   10|
| 26|   17|
| 27|    8|
| 44|   12|
| 22|    7|
| 47|    9|
+---+-----+
only showing top 10 rows



In [34]:
output.createOrReplaceTempView("peoples")

In [31]:
spark.sql("select * from peoples").show(100)

+------+--------+---+-------+--------------------+
|userID|    name|age|friends|           insert_ts|
+------+--------+---+-------+--------------------+
|     1|Jean-Luc| 26|      2|2024-07-05 14:01:...|
|     9|    Hugh| 27|    181|2024-07-05 14:01:...|
|    16|  Weyoun| 22|    323|2024-07-05 14:01:...|
|    21|   Miles| 19|    268|2024-07-05 14:01:...|
|    24|  Julian| 25|      1|2024-07-05 14:01:...|
|    25|     Ben| 21|    445|2024-07-05 14:01:...|
|    26|  Julian| 22|    100|2024-07-05 14:01:...|
|    32|     Nog| 26|    281|2024-07-05 14:01:...|
|    35| Beverly| 27|    305|2024-07-05 14:01:...|
|    46|    Morn| 25|     96|2024-07-05 14:01:...|
|    47|   Brunt| 24|     49|2024-07-05 14:01:...|
|    48|     Nog| 20|      1|2024-07-05 14:01:...|
|    52| Beverly| 19|    269|2024-07-05 14:01:...|
|    54|   Brunt| 19|      5|2024-07-05 14:01:...|
|    60|  Geordi| 20|    100|2024-07-05 14:01:...|
|    66|  Geordi| 21|    477|2024-07-05 14:01:...|
|    72|  Kasidy| 22|    179|20

In [28]:
spark.sql("select name,age,friends from peoples").show()

+--------+---+-------+
|    name|age|friends|
+--------+---+-------+
|Jean-Luc| 26|      2|
|    Hugh| 27|    181|
|  Weyoun| 22|    323|
|   Miles| 19|    268|
|  Julian| 25|      1|
|     Ben| 21|    445|
|  Julian| 22|    100|
|     Nog| 26|    281|
| Beverly| 27|    305|
|    Morn| 25|     96|
|   Brunt| 24|     49|
|     Nog| 20|      1|
| Beverly| 19|    269|
|   Brunt| 19|      5|
|  Geordi| 20|    100|
|  Geordi| 21|    477|
|  Kasidy| 22|    179|
|   Brunt| 20|    384|
|     Ben| 28|    311|
|    Worf| 24|    492|
+--------+---+-------+
only showing top 20 rows

